In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import os
from skimage import io
from PIL import Image
import csv
import pandas as pd
from keras.models import load_model
from keras import backend as K
import cv2

In [ ]:
#加载模型
K.clear_session()
model = load_model('D:/DP.h5')

In [ ]:
# 读取原图
filenames=os.listdir('D:/pic1/')


def read_image2(filename):
    img = Image.open(('D:/pic1/')+filename).convert('RGB')
    return np.array(img)

y = []

for i in filenames:  
    y.append(read_image2(i))

y = np.array(y)

In [ ]:
y = y.astype('float32')
y_normalize = y / 255

In [ ]:
#预测原图性能
y_pred = model.predict(y_normalize)
print(y_pred)

In [ ]:
#生成原图对应的可视化热图
pic_output = model.output
last_conv_layer= model.get_layer('conv2d_7')
grads = K.gradients(pic_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis=(0, 1, 2))
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([y_normalize])
for i in range(64):     
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i] 
heatmap = np.mean(conv_layer_output_value, axis=-1) 
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

In [ ]:
#将可视化热图与原图叠放
img = cv2.imread('D:/pic1/1.tif')
heatmap = cv2.resize(heatmap, (336, 336)) 
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET) 
superimposed_img = heatmap * 0.4 + img
cv2.imwrite('D:/pic1/2.tif',superimposed_img ) 